In [2]:
# Pipeline的使用
# pipeline 是一种设计方式，将模型的输入、输出、预处理、后处理等步骤封装在一起，方便使用。

from transformers.pipelines import SUPPORTED_TASKS

In [5]:
for k,v in SUPPORTED_TASKS.items():
    print(k,v["type"])

audio-classification audio
automatic-speech-recognition multimodal
text-to-audio text
feature-extraction multimodal
text-classification text
token-classification text
question-answering text
table-question-answering text
visual-question-answering multimodal
document-question-answering multimodal
fill-mask text
summarization text
translation text
text2text-generation text
text-generation text
zero-shot-classification text
zero-shot-image-classification multimodal
zero-shot-audio-classification multimodal
image-classification image
image-feature-extraction image
image-segmentation multimodal
image-to-text multimodal
image-text-to-text multimodal
object-detection multimodal
zero-shot-object-detection multimodal
depth-estimation image
video-classification video
mask-generation multimodal
image-to-image image


In [6]:
# Pipeline的创建和使用方式
from transformers import pipeline


In [13]:
# 1. 根据任务类型直接创建Pipeline，默认都是英文模型
pipe = pipeline("text-classification")

result = pipe("like you")
print(result)



No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


[{'label': 'POSITIVE', 'score': 0.999830961227417}]


In [16]:
# 2. 根据任务类型和模型名称创建Pipeline
pipe = pipeline("text-classification", model="uer/roberta-base-finetuned-dianping-chinese")

result = pipe("I love you")
print(result)

Device set to use mps:0


[{'label': 'positive (stars 4 and 5)', 'score': 0.9009531140327454}]


/Users/wuhaoliu/micromamba/envs/transformers/lib/python3.9/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [18]:
result = pipe("今天天气真差劲呀！")
print(result)

[{'label': 'negative (stars 1, 2 and 3)', 'score': 0.9818915724754333}]


/Users/wuhaoliu/micromamba/envs/transformers/lib/python3.9/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [22]:
# 3. 预先加载模型和分词器，再创建 pipeline
from transformers import AutoModelForSequenceClassification,AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("uer/roberta-base-finetuned-dianping-chinese")
tokenizer = AutoTokenizer.from_pretrained("uer/roberta-base-finetuned-dianping-chinese")

pipe=pipeline("text-classification",model=model,tokenizer=tokenizer)

result=pipe("今天天气真差劲呀！")
print(result)

Device set to use mps:0


[{'label': 'negative (stars 1, 2 and 3)', 'score': 0.9818915724754333}]


/Users/wuhaoliu/micromamba/envs/transformers/lib/python3.9/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [23]:
pipe=pipeline("text-classification",model=model)

result=pipe("今天天气真差劲呀！")
print(result)

Exception: Impossible to guess which tokenizer to use. Please provide a PreTrainedTokenizer class or a path/identifier to a pretrained tokenizer.